In [1]:
import os

os.chdir('..')

import pandas as pd
import numpy as np
import shutil

from inference_utils import *

In [2]:
os.getcwd()

'/home/jp4474/latent-abc-smc'

In [3]:
# Find all folders named lotka_d64
lotka_folders = [f for f in os.listdir('.') if f.startswith('lotka_d64') and os.path.isdir(f)]

In [4]:
# From those folders, find npz files
npz_files = []
for folder in lotka_folders:
    for file in os.listdir(folder):
        if file.endswith('.npz'):
            npz_files.append(os.path.join(folder, file))

In [5]:
# Load the npz files, extract generations and the last generation
data = []
for npz_file in npz_files:
    with np.load(npz_file, allow_pickle=True) as data_npz:
        generations = data_npz['generations']
        last_generation = generations[-1]
        # Calculate the mean and std of the last generation
        mean = np.average(last_generation['particles'], weights=last_generation['weights'], axis=0)
        var = np.average((last_generation['particles'] - mean) ** 2, weights=last_generation['weights'], axis=0)
        data.append({
            'model': npz_file.split('/')[0],
            'idx': npz_file.split('/')[-1].split('_')[-1].replace('.npz', ''),
            'mean': mean,
            'std': var
        })


In [6]:
df = pd.DataFrame(data)
df['idx'] = df['idx'].replace('particles1000', '1')
df['mean_a'] = df['mean'].apply(lambda x: x[0])
df['mean_b'] = df['mean'].apply(lambda x: x[1])
df['std_a'] = df['std'].apply(lambda x: x[0])
df['std_b'] = df['std'].apply(lambda x: x[1])

In [7]:
df.head()   

,model,idx,mean,std,mean_a,mean_b,std_a,std_b
0,lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...,5,"[1.0862667012869847, 0.9038883221763886]","[8.585490513976989e-05, 0.00031146504137340587]",1.086267,0.903888,0.000086,0.000311
1,lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...,4,"[1.0886394276346896, 0.8994720199931509]","[4.7844470088782914e-05, 0.00016186972989276215]",1.088639,0.899472,0.000048,0.000162
2,lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...,3,"[1.0602323605206174, 0.9864577338744608]","[0.011899422367906008, 0.04577303865058321]",1.060232,0.986458,0.011899,0.045773
3,lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...,2,"[1.0889920457833773, 0.8987376322034306]","[4.304310027079857e-05, 0.00015127568889506697]",1.088992,0.898738,0.000043,0.000151
4,lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...,1,"[1.08750591192987, 0.9014898082257866]","[6.093862870960751e-05, 0.00021694032865901718]",1.087506,0.901490,0.000061,0.000217


In [8]:
# take the mean for mean while 
df[['model', 'mean_a', 'mean_b']].groupby(['model']).aggregate(['mean', 'std']).reset_index()

model    mean_a            \
                                                           mean       std   
0       lotka_d64_ed32_6_4_4_4_ae_mask_0.15_noise_0.0  1.156450  0.043800   
1       lotka_d64_ed32_6_4_4_4_ae_mask_0.15_noise_0.5  1.034220  0.027270   
2       lotka_d64_ed32_6_4_4_4_ae_mask_0.15_noise_1.0  0.915516  0.000941   
3   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  1.267600  0.199020   
4   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  1.082327  0.012397   
5   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  0.986196  0.018161   
6   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  1.266174  0.147077   
7   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  1.014129  0.026714   
8   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.00...  0.966085  0.005645   
9   lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.01...  1.154277  0.065652   
10  lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.01...  1.005948  0.081380   
11  lotka_d64_ed32_6_4_4_4_vae_mask_0.15_beta_0.01...  0.953594  0.003986   

      mean_b            
        mean       std  
0   1.090065  0.201564  
1   1.077773  0.068412  
2   1.480386  0.004563  
3   0.845158  0.378798  
4   0.918009  0.038316  
5   1.168961  0.026392  
6   0.819505  0.262532  
7   1.115979  0.124174  
8   1.219556  0.009321  
9   2.201568  0.601344  
10  0.995438  0.061750  
11  1.289168  0.012095